In [1]:
sc

''

In [2]:
import numpy as np
from os import listdir
from os.path import join, basename
import struct
from scipy import misc
from transformer import Resize

#Import all the required packages

%pylab inline

import pandas
import datetime as dt

#Importing necessary packages 
from bigdl.nn.layer import *
from optparse import OptionParser
from bigdl.nn.criterion import *
from bigdl.optim.optimizer import *
from bigdl.util.common import *
from bigdl.dataset.transformer import *
from bigdl.nn.initialization_method import *
from transformer import *
from imagenet import *
#from bigdl.dataset import imagenet



def read_local_path(folder, has_label=True):
    # read directory, create map
    dirs = listdir(folder)
    print folder
    print dirs
    # create image path and label list
    image_paths = []
    print image_paths
    if has_label:
        dirs.sort()
        for d in dirs:
            for f in listdir(join(folder, d)):
                image_paths.append((join(join(folder, d), f), dirs.index(d) + 1))
    else:
        for f in dirs:
            image_paths.append((join(folder, f), -1))
    return image_paths


Populating the interactive namespace from numpy and matplotlib


In [3]:
def read_local(sc, folder, normalize=255.0, has_label=True):
    """
    Read images from local directory
    :param sc: spark context
    :param folder: local directory
    :param normalize: normalization value
    :param has_label: whether the image folder contains label
    :return: RDD of sample
    """
    # read directory, create image paths list
    image_paths = read_local_path(folder, has_label)
    print image_paths
    # create rdd
    image_paths_rdd = sc.parallelize(image_paths)
    print image_paths_rdd
    feature_label_rdd = image_paths_rdd.map(lambda path_label: (misc.imread(path_label[0]), np.array(path_label[1]))) \
        .map(lambda img_label:
             (Resize(256, 256)(img_label[0]), img_label[1])) \
        .map(lambda feature_label:
             (((feature_label[0] & 0xff) / normalize).astype("float32"), feature_label[1]))
    return feature_label_rdd

The following function takes an input, if the input is a list, it insert into index 0 spot, such that the real data starts from index 1. Returns back a dictionary with key being the index and value the list of numbers.

In [4]:
def scala_T(input_T):
    
    if type(input_T) is list:
        # insert into index 0 spot, such that the real data starts from index 1
        temp = [0]
        temp.extend(input_T)
        return dict(enumerate(temp))
    # if dictionary, return it back
    return input_T

The following codes basically creates functions that initiate the inception model. Each function is independent of the other, the architecture is what makes these functions unique. 

In [5]:
def Inception_Layer_v1(input_size, config, name_prefix=""):
    
    
    '''
    Concat is a container who concatenates the output of it's submodules along the provided dimension: all submodules 
    take the same inputs, and their output is concatenated.
    '''
    concat = Concat(2)
    
    """
    In the above code, we first create a container Sequential. Then add the layers into the container one by one. The 
    order of the layers in the model is same with the insertion order. 
    
    """
    conv1 = Sequential()
    
    #Adding layes to the conv1 model we jus created
    
    #SpatialConvolution is a module that applies a 2D convolution over an input image.
    conv1.add(SpatialConvolution(input_size, config[1][1], 1, 1, 1, 1).set_name(name_prefix + "1x1"))
    conv1.add(ReLU(True).set_name(name_prefix + "relu_1x1"))
    concat.add(conv1)
    
    conv3 = Sequential()
    conv3.add(SpatialConvolution(input_size, config[2][1], 1, 1, 1, 1).set_name(name_prefix + "3x3_reduce"))
    conv3.add(ReLU(True).set_name(name_prefix + "relu_3x3_reduce"))
    conv3.add(SpatialConvolution(config[2][1], config[2][2], 3, 3, 1, 1, 1, 1).set_name(name_prefix + "3x3"))
    conv3.add(ReLU(True).set_name(name_prefix + "relu_3x3"))
    concat.add(conv3)
    
    
    conv5 = Sequential()
    conv5.add(SpatialConvolution(input_size,config[3][1], 1, 1, 1, 1).set_name(name_prefix + "5x5_reduce"))
    conv5.add(ReLU(True).set_name(name_prefix + "relu_5x5_reduce"))
    conv5.add(SpatialConvolution(config[3][1], config[3][2], 5, 5, 1, 1, 2, 2).set_name(name_prefix + "5x5"))
    conv5.add(ReLU(True).set_name(name_prefix + "relu_5x5"))
    concat.add(conv5)
    
    
    pool = Sequential()
    pool.add(SpatialMaxPooling(3, 3, 1, 1, 1, 1, to_ceil=True).set_name(name_prefix + "pool"))
    pool.add(SpatialConvolution(input_size, config[4][1], 1, 1, 1, 1).set_name(name_prefix + "pool_proj"))
    pool.add(ReLU(True).set_name(name_prefix + "relu_pool_proj"))
    concat.add(pool).set_name(name_prefix + "output")
    return concat

In [6]:
def Inception_v1(class_num):
    feature1 = Sequential()
    
    feature1.add(SpatialConvolution(3, 64, 7, 7, 2, 2, 3, 3, 1, False).set_name("conv1/7x7_s2"))
    feature1.add(ReLU(True).set_name("conv1/relu_7x7"))
    feature1.add(SpatialMaxPooling(3, 3, 2, 2, to_ceil=True).set_name("pool1/3x3_s2"))
    feature1.add(SpatialCrossMapLRN(5, 0.0001, 0.75).set_name("pool1/norm1"))
    feature1.add(SpatialConvolution(64, 64, 1, 1, 1, 1).set_name("conv2/3x3_reduce"))
    feature1.add(ReLU(True).set_name("conv2/relu_3x3_reduce"))
    feature1.add(SpatialConvolution(64, 192, 3, 3, 1, 1, 1, 1).set_name("conv2/3x3"))
    feature1.add(ReLU(True).set_name("conv2/relu_3x3"))
    feature1.add(SpatialCrossMapLRN(5, 0.0001, 0.75).set_name("conv2/norm2"))
    feature1.add(SpatialMaxPooling(3, 3, 2, 2, to_ceil=True).set_name("pool2/3x3_s2"))
    feature1.add(Inception_Layer_v1(192,scala_T([scala_T([64]), scala_T([96, 128]),scala_T([16, 32]), scala_T([32])]),
                                    "inception_3a/"))
    feature1.add(Inception_Layer_v1(256, scala_T([scala_T([128]), scala_T([128, 192]), scala_T([32, 96]), scala_T([64])]),
                                    "inception_3b/"))
    feature1.add(SpatialMaxPooling(3, 3, 2, 2, to_ceil=True).set_name("pool3/3x3_s2"))
    feature1.add(Inception_Layer_v1(480, scala_T([scala_T([192]), scala_T([96, 208]), scala_T([16, 48]), scala_T([64])]),
                                    "inception_4a/"))

    output1 = Sequential()
    output1.add(SpatialAveragePooling(5, 5, 3, 3, ceil_mode=True).set_name("loss1/ave_pool"))
    output1.add(SpatialConvolution(512, 128, 1, 1, 1, 1).set_name("loss1/conv"))
    output1.add(ReLU(True).set_name("loss1/relu_conv"))
    output1.add(View([128 * 4 * 4], num_input_dims = 3))
    output1.add(Linear(128 * 4 * 4, 1024).set_name("loss1/fc"))
    output1.add(ReLU(True).set_name("loss1/relu_fc"))
    output1.add(Dropout(0.7).set_name("loss1/drop_fc"))
    output1.add(Linear(1024, class_num).set_name("loss1/classifier"))
    output1.add(LogSoftMax().set_name("loss1/loss"))

    
    feature2 = Sequential()
    feature2.add(Inception_Layer_v1(512, scala_T([scala_T([160]), scala_T([112, 224]),scala_T([24, 64]), scala_T([64])]),
                                    "inception_4b/"))
    feature2.add(Inception_Layer_v1(512, scala_T([scala_T([128]), scala_T([128, 256]),scala_T([24, 64]), scala_T([64])]),
                                    "inception_4c/"))
    feature2.add(Inception_Layer_v1(512, scala_T([scala_T([112]), scala_T([144, 288]), scala_T([32, 64]), scala_T([64])]),
                                    "inception_4d/"))

    output2 = Sequential()
    output2.add(SpatialAveragePooling(5, 5, 3, 3).set_name("loss2/ave_pool"))
    output2.add(
        SpatialConvolution(528, 128, 1, 1, 1, 1).set_name("loss2/conv"))
    output2.add(ReLU(True).set_name("loss2/relu_conv"))
    output2.add(View([128 * 4 * 4], num_input_dims=3))
    output2.add(Linear(128 * 4 * 4, 1024).set_name("loss2/fc"))
    output2.add(ReLU(True).set_name("loss2/relu_fc"))
    output2.add(Dropout(0.7).set_name("loss2/drop_fc"))
    output2.add(Linear(1024, class_num).set_name("loss2/classifier"))
    output2.add(LogSoftMax().set_name("loss2/loss"))

    output3 = Sequential()
    output3.add(Inception_Layer_v1(528, scala_T([scala_T([256]), scala_T([160, 320]), scala_T([32, 128]), scala_T([128])]),
                                   "inception_4e/"))
    output3.add(SpatialMaxPooling(3, 3, 2, 2, to_ceil=True).set_name("pool4/3x3_s2"))
    output3.add(Inception_Layer_v1(832, scala_T([scala_T([256]), scala_T([160, 320]), scala_T([32, 128]), scala_T([128])]),
                                   "inception_5a/"))
    output3.add(Inception_Layer_v1(832,scala_T([scala_T([384]), scala_T([192, 384]),scala_T([48, 128]), scala_T([128])]),
                                   "inception_5b/"))
    output3.add(SpatialAveragePooling(7, 7, 1, 1).set_name("pool5/7x7_s1"))
    output3.add(Dropout(0.4).set_name("pool5/drop_7x7_s1"))
    output3.add(View([1024], num_input_dims=3))
    output3.add(Linear(1024, class_num).set_name("loss3/classifier"))
    output3.add(LogSoftMax().set_name("loss3/loss3"))

    split2 = Concat(2).set_name("split2")
    split2.add(output3)
    split2.add(output2)

    mainBranch = Sequential()
    mainBranch.add(feature2)
    mainBranch.add(split2)

    split1 = Concat(2).set_name("split1")
    split1.add(mainBranch)
    split1.add(output1)

    model = Sequential()

    model.add(feature1)
    model.add(split1)

    model.reset()
    return model

In [7]:
def get_inception_data(folder, file_type="image", data_type="train", normalize=255.0):
    
#   Getting the path of our data
    path = os.path.join(folder, data_type)

    if "seq" == file_type:
#         return imagenet.read_seq_file(sc, path, normalize)
        return read_seq_file(sc, path, normalize)
    elif "image" == file_type:
#         return imagenet.read_local(sc, path, normalize)
        return read_local(sc, path, normalize)

In [8]:
def config_option_parser():
    parser = OptionParser()
    parser.add_option("-a", "--action", dest="action", default="train")
    parser.add_option("-f", "--folder", type=str, dest="folder", default="",
                      help="url of hdfs folder store the hadoop sequence files")
    parser.add_option("--model", type=str, dest="model", default="", help="model snapshot location")
    parser.add_option("--state", type=str, dest="state", default="", help="state snapshot location")
    parser.add_option("--checkpoint", type=str, dest="checkpoint", default="", help="where to cache the model")
    parser.add_option("-o", "--overwrite", action="store_true", dest="overwrite", default=False,
                      help="overwrite checkpoint files")
    parser.add_option("-e", "--maxEpoch", type=int, dest="maxEpoch", default=0, help="epoch numbers")
    parser.add_option("-i", "--maxIteration", type=int, dest="maxIteration", default=62000, help="iteration numbers")
    parser.add_option("-l", "--learningRate", type=float, dest="learningRate", default=0.01, help="iteration numbers")
    parser.add_option("-b", "--batchSize", type=int, dest="batchSize", help="batch size")
    parser.add_option("--classNum", type=int, dest="classNum", default=1000, help="class number")
    parser.add_option("--weightDecay", type=float, dest="weightDecay", default=0.0001, help="weight decay")
    parser.add_option("--checkpointIteration", type=int, dest="checkpointIteration", default=620,
                      help="checkpoint interval of iterations")

    return parser

In [9]:
parser = config_option_parser()
(options, args) = parser.parse_args(sys.argv)

# if not options.learningRate:
#     parser.error("-l --learningRate is a mandatory opt")
# if not options.batchSize:
#     parser.error("-b --batchSize is a mandatory opt")

In [10]:
#init engine
init_engine()

# Build the Model
inception_model = Inception_v1(options.classNum)

creating: createSequential
creating: createSpatialConvolution
creating: createReLU
creating: createSpatialMaxPooling
creating: createSpatialCrossMapLRN
creating: createSpatialConvolution
creating: createReLU
creating: createSpatialConvolution
creating: createReLU
creating: createSpatialCrossMapLRN
creating: createSpatialMaxPooling
creating: createConcat
creating: createSequential
creating: createSpatialConvolution
creating: createReLU
creating: createSequential
creating: createSpatialConvolution
creating: createReLU
creating: createSpatialConvolution
creating: createReLU
creating: createSequential
creating: createSpatialConvolution
creating: createReLU
creating: createSpatialConvolution
creating: createReLU
creating: createSequential
creating: createSpatialMaxPooling
creating: createSpatialConvolution
creating: createReLU
creating: createConcat
creating: createSequential
creating: createSpatialConvolution
creating: createReLU
creating: createSequential
creating: createSpatialConvolutio

In [ ]:
image_size = 224
batch_size = 16
classNum = 2
no_epochs = 2
options.action = "train"
if options.action == "train":
    
    # create dataset
    train_transformer = Transformer([Crop(image_size, image_size),
                                     Flip(0.5),
                                     ChannelNormalizer(0.485, 0.456, 0.406, 0.229, 0.224, 0.225),
                                     TransposeToTensor(False)
                                     ])
    
    options.folder = "../sample/"
#     print options.folder
    train_data = get_inception_data(options.folder, "image", "train").map(
        lambda features_label: (train_transformer(features_label[0]), features_label[1])).map(
        lambda features_label: Sample.from_ndarray(features_label[0], features_label[1] + 1))
        
    val_transformer = Transformer([Crop(image_size, image_size, "center"),
                                   Flip(0.5),
                                   ChannelNormalizer(0.485, 0.456, 0.406, 0.229, 0.224, 0.225),
                                   TransposeToTensor(False)
                                   ])
    #val_data = get_inception_data(options.folder, "seq", "val").map(
    val_data = get_inception_data(options.folder, "image", "val").map(
        lambda features_label: (val_transformer(features_label[0]), features_label[1])).map(
        lambda features_label: Sample.from_ndarray(features_label[0], features_label[1] + 1))

    # TODO: Check stateSnapshot opt

    if options.maxEpoch:
            {"learningRate": options.learningRate,
             "weightDecay": options.weightDecay,
             "momentum": 0.9,
             "dampening": 0.0}
            checkpoint_trigger = EveryEpoch()
            test_trigger = EveryEpoch()
            end_trigger = MaxEpoch(options.maxEpoch)
    else:
        state = scala_T(
            {"learningRate": options.learningRate,
             "weightDecay": options.weightDecay,
             "momentum": 0.9,
             "dampening": 0.0})
        checkpoint_trigger = SeveralIteration(options.checkpointIteration)
        test_trigger = SeveralIteration(options.checkpointIteration)
        end_trigger = MaxIteration(10)

    # Optimizer
    optimizer = Optimizer(
        model=inception_model,
        training_rdd=train_data,
#         optim_method="SGD",
        optim_method = SGD(learningrate=0.01, learningrate_decay=0.0002),
#         optim_method=Adam(learningrate=0.002),
        criterion=ClassNLLCriterion(),
        end_trigger=end_trigger,
        batch_size=batch_size,
        #state=state
        )
    
    # Optimizer
#     optimizer = Optimizer(
#                 model=inception_model,
#                 training_rdd=train_data,
#                 optim_method=Adam(learningrate=0.002),
#                 criterion=ClassNLLCriterion(),
#                 end_trigger=MaxEpoch(no_epochs),
#                 batch_size=batch_size
#                 )
        
    options.batchSize = 64 
    
    options.checkpoint = "../Final_MATT/"
    if options.checkpoint:
        optimizer.set_checkpoint(checkpoint_trigger, options.checkpoint, options.overwrite)

    """
    optimizer.set_validation(trigger=test_trigger,
                             val_rdd=val_data,
                             batch_size=options.batchSize,
                             #val_method=["Top1Accuracy", "Top5Accuracy"])
                             val_method=["Loss"]
                             val_method=[Top1Accuracy()])
    """
    optimizer.set_validation(batch_size=2048,
                            val_rdd=val_data,
                            trigger=EveryEpoch(),
                            val_method=[Top1Accuracy()])

    trained_model = optimizer.optimize()
    
elif options.action == "test":
    # Load a pre-trained model and then validate it through top1 accuracy.
    test_transformer = Transformer([Crop(image_size, image_size, "center"),
                                    Flip(0.5),
                                    ChannelNormalizer(0.485, 0.456, 0.406, 0.229, 0.224, 0.225),
                                    TransposeToTensor(False)
                                    ])
    test_data = get_inception_data(options.folder, "seq", "val").map(
        lambda features_label: (test_transformer(features_label[0]), features_label[1])).map(
        lambda features_label: Sample.from_ndarray(features_label[0], features_label[1] + 1))
    model = Model.load(options.model)
    results = model.test(test_data, options.batchSize, ["Top1Accuracy", "Top5Accuracy"])
    for result in results:
        print result

../sample/train
['nonveg', 'veg']
[]
[('../sample/train/nonveg/n07711080_1284.JPEG', 1), ('../sample/train/nonveg/n07711080_2950.JPEG', 1), ('../sample/train/nonveg/n07711080_757.JPEG', 1), ('../sample/train/veg/n07705931_3672.JPEG', 2), ('../sample/train/veg/n07705931_6350.JPEG', 2), ('../sample/train/veg/n07705931_76.JPEG', 2)]
ParallelCollectionRDD[107] at parallelize at PythonRDD.scala:475
../sample/val
['nonveg', 'veg']
[]
[('../sample/val/nonveg/n07710616_21595.JPEG', 1), ('../sample/val/nonveg/n07710616_32372.JPEG', 1), ('../sample/val/nonveg/n07710616_52269.JPEG', 1), ('../sample/val/veg/n00017222_17493.JPEG', 2), ('../sample/val/veg/n00017222_17592.JPEG', 2), ('../sample/val/veg/n00017222_5826.JPEG', 2)]
ParallelCollectionRDD[108] at parallelize at PythonRDD.scala:475
creating: createSeveralIteration
creating: createSeveralIteration
creating: createMaxIteration
creating: createDefault
creating: createSGD
creating: createClassNLLCriterion
creating: createOptimizer
creating: cre